In [2]:
import google.generativeai as genai
import os
import re
import time
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()  

class GeneradorClases:
    """
    Genera clases Python usando la API de Google Gemini.
    Encapsula la configuración, el prompt y la comunicación con la IA.
    """

    def __init__(self, modelo="gemini-2.5-flash"):
        # Leer la API Key de variable de entorno (seguro)
        api_key = os.environ.get("GEMINI_API_KEY")

        if not api_key:
            raise ValueError(
                "No se encontró GEMINI_API_KEY. "
                "Configúrala con: export GEMINI_API_KEY='tu_clave'"
            )

        # Configurar la librería con la clave
        genai.configure(api_key=api_key)

        # Crear el modelo con instrucción de sistema
        self.model = genai.GenerativeModel(
            model_name=modelo,
            system_instruction=(
                "Eres un profesor de Python para estudiantes "
                "de estadística en la Universidad Santo Tomás. "
                "Genera SOLO código Python puro, sin markdown."
            )
        )
        self.intentos_max = 3

    def generar(self, concepto):
        """Genera una clase Python para el concepto dado."""
        prompt = f"""Genera una clase Python para: "{concepto}".
Incluye:
- Docstring descriptivo
- Constructor __init__ con al menos 3 atributos
- Al menos 2 métodos útiles con docstrings
- Comentarios en español
- Ejemplo de uso al final"""

        # Reintentos con backoff exponencial
        for intento in range(self.intentos_max):
            try:
                respuesta = self.model.generate_content(prompt)
                codigo = respuesta.text

                # Limpiar posibles bloques markdown
                codigo = re.sub(
                    r'^\x60\x60\x60python\n?', '', codigo
                )
                codigo = re.sub(r'\n?\x60\x60\x60$', '', codigo)
                return codigo.strip()

            except Exception as e:
                print(f"Intento {intento + 1} falló: {e}")
                if intento < self.intentos_max - 1:
                    espera = 2 ** intento  # 1s, 2s, 4s
                    print(f"Reintentando en {espera}s...")
                    time.sleep(espera)
                else:
                    return f"Error después de {self.intentos_max} intentos: {e}"

# --- Uso del generador ---
if __name__ == "__main__":
    generador = GeneradorClases()

    concepto = input("¿Qué clase quieres generar? ")
    print(f"\nGenerando clase para '{concepto}'...\n")

    resultado = generador.generar(concepto)
    print(resultado)


Generando clase para 'camisa'...

class Camisa:
    """
    Representa una prenda de vestir tipo camisa con sus características principales.
    Esta clase permite definir una camisa por su talla, color, material y marca,
    además de ofrecer métodos para interactuar con sus propiedades.
    """

    def __init__(self, talla, color, material, marca):
        """
        Constructor de la clase Camisa. Inicializa una nueva instancia de camisa.

        Args:
            talla (str): La talla de la camisa (ej. "S", "M", "L", "XL").
            color (str): El color de la camisa (ej. "blanco", "azul", "negro").
            material (str): El material principal de la camisa (ej. "algodón", "lino", "seda").
            marca (str): La marca de la camisa (ej. "Zara", "H&M", "Levi's").
        """
        # Atributos de la camisa
        self.talla = talla  # Almacena la talla de la camisa
        self.color = color  # Almacena el color actual de la camisa
        self.material = material  

In [1]:
import os
from google import genai

# Inicializamos el cliente de Gemini.
# Por defecto, buscará la llave en la variable de entorno GEMINI_API_KEY.
# También puedes pasarla directamente: genai.Client(api_key="TU_LLAVE_AQUI")
client = genai.Client()

def generar_modelo_pydantic(datos_crudos: str) -> str:
    """
    Toma un string con datos (JSON o descripción) y usa Gemini
    para generar el código de un modelo Pydantic validado.
    """
    
    # Este es el mismo "Prompt" que usamos en la versión web
    prompt = f"""
    Actúa como un tutor experto en Data Science con Python.
    Tengo estos datos crudos (JSON o descripción):
    ---
    {datos_crudos}
    ---
    Por favor, genera un modelo completo de Pydantic en Python (heredando de BaseModel) para validar y limpiar estos datos.
    1. Usa los tipos adecuados (int, float, str, EmailStr si aplica).
    2. Usa 'Field' para descripciones y restricciones numéricas.
    3. Si los datos parecen "sucios" (ej. "25 años" para un int), agrega un @field_validator con mode='before' para limpiarlo.
    4. Agrega comentarios en español explicando la lógica.
    5. Provee un pequeño ejemplo de uso al final.
    Devuelve ÚNICAMENTE el bloque de código Python listo para ejecutarse.
    """

    print("Analizando datos y generando modelo con Gemini... ✨")
    
    # Llamada a la API de Gemini usando el modelo más rápido y eficiente
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=prompt,
    )
    
    # Extraemos el texto de la respuesta
    codigo = response.text
    
    # Limpiamos los bloques de código Markdown (```python) que suele devolver la IA
    codigo = codigo.replace("```python", "").replace("```", "").strip()
    
    return codigo

# --- Ejemplo de Uso ---
if __name__ == "__main__":
    # Simulemos algunos datos crudos que copiamos de un Excel o API
    mis_datos_sucios = """
    {
      "nombre_paciente": "Ana Martinez",
      "edad": "34 años",
      "email": "ana.m@hospital.com",
      "nivel_glucosa": "105,5",
      "fumador": "sí"
    }
    """
    
    try:
        # 1. Generamos el código
        codigo_pydantic = generar_modelo_pydantic(mis_datos_sucios)
        
        # 2. Mostramos el resultado
        print("\n" + "="*50)
        print("🤖 CÓDIGO PYDANTIC GENERADO POR AI:")
        print("="*50 + "\n")
        print(codigo_pydantic)
        print("\n" + "="*50)
        
        # Opcional: Podrías guardar este código en un archivo directamente
        # with open("modelos_generados.py", "w", encoding="utf-8") as f:
        #     f.write(codigo_pydantic)
            
    except Exception as e:
        print(f"❌ Ocurrió un error al conectar con Gemini: {e}")
        print("Asegúrate de haber configurado tu GEMINI_API_KEY correctamente.")

Analizando datos y generando modelo con Gemini... ✨

🤖 CÓDIGO PYDANTIC GENERADO POR AI:

from pydantic import BaseModel, Field, EmailStr, ValidationError, field_validator
from typing import Literal # Se importa aunque no se use directamente para 'fumador' con la estrategia actual, es común en Pydantic.

# Modelo Pydantic para validar y limpiar los datos del paciente.
class DatosPaciente(BaseModel):
    """
    Modelo Pydantic para la validación y limpieza de datos de pacientes.
    Hereda de BaseModel para proporcionar funcionalidades de validación de datos.
    """

    # Campo: nombre_paciente
    # Tipo: str
    # Descripción: Nombre completo del paciente.
    # No requiere limpieza especial, Pydantic lo manejará como string.
    nombre_paciente: str = Field(
        ..., # Indica que el campo es obligatorio
        description="Nombre completo del paciente."
    )

    # Campo: edad
    # Tipo: int
    # Descripción: Edad del paciente en años.
    # Restricciones: Debe ser mayor qu